# Econ 8210 Quant Macro, Homework 1
## Part 2 - Solution Methods
Haosi Shen, Fall 2024

In [5]:
# Housekeeping
import numpy as np
import pandas as pd
import time
import torch

np.random.seed(42) 

## Computing Pareto Efficient Allocations

Consider an endowment economy with $m$ different goods and $n$ agents. Each agent $i = 1, ..., n$ has an endowment $e_j^i >0$ for every $j = 1, ..., m$ and a utility function of the form
$$ u^i (x) = \sum_{j=1}^{m} \alpha_j \frac{x_{j}^{1+\omega_j^i}}{1+\omega_j^i} $$
where $\alpha_j > 0 > \omega_j^i$ are agent-specific parameters.

Given some social weights $\lambda_i > 0$, solve for the social planner’s problem for $m = n = 3$ using the **Adam (Adaptive Moment Estimation)** method. Try different values of $\alpha_j,\;\omega_j^i,\;\lambda_i$. 

$$ \max_{\{x^i\}} \; \sum_{i=1}^{n} \lambda_i u^{i}(x)$$

Compute first the case where all the agents have the same parameters and
social weights and later a case where there is a fair degree of heterogeneity.

How does the method perform? How does heterogeneity in the agent-specific parameters
affect the results?

Can you handle the case where $m = n = 10$?

> I choose to use **Adam** since it is more efficient for high-dimensional optimization problems and offers more stability and robustness. However, if we are only solving for the case of $m=n=3$, then the Newton-Raphson method might be more ideal since this problem is relatively smooth. Adam only requires gradient information and does not involve inverting the Hessian.

In [17]:
# utility function for agent i
def utility(x, alpha, omega):
    return torch.sum(alpha * (x ** (1 + omega)) / (1 + omega))


# Incorporate resource constraints for each good j
# Define total endowments
endowments = torch.tensor([30.0, 30.0, 30.0], dtype = torch.float32)

### Case I: Homogeneous Agents

> $m = n = 3$

All agents $j$ have the same parameters $\alpha_j, \omega_j^i$ and social weights $\lambda_j$. 

In [18]:
# Social planner's objective, with Homogeneous Agents
def social_planner_objective(x, alpha_j, omega_j, lambda_i, endowments, penalty_weight = 1000):
    total_utility = 0
    total_allocations = torch.zeros(m)

    for i in range(n):
        x_i_softplus = torch.nn.functional.softplus(x[i]) # ensure nonneg allocations
        total_utility += lambda_i[i] * utility(x_i_softplus, alpha_j, omega_j)
        total_allocations += x_i_softplus

    # penalize if RC is violated, i.e. total allocations > endowments
    penalty = torch.sum(torch.clamp(total_allocations - endowments, min = 0) ** 2)

    return -total_utility + penalty_weight * penalty   # maximization, so take negative

In [43]:
import torch.optim as optim

# =============== DEFINE PARAMETERS ===============
m, n = 3, 3  # 3 goods, 3 agents
alpha_j = torch.tensor([1.0, 1.0, 1.0], dtype = torch.float32) 
omega_j = torch.tensor([[0.5, 0.5, 0.5]] * n, dtype = torch.float32)
lambda_i = torch.tensor([1.0, 1.0, 1.0], dtype = torch.float32)  # Pareto weights

# initial allocations (using small positive values)
x_i = torch.rand((n, m), requires_grad = True)

# Set up optimizer
optimizer = optim.Adam([x_i], lr = 0.01)

# Optimization
num_iterations = 1000
for iteration in range(num_iterations):
    optimizer.zero_grad()
    objective = social_planner_objective(x_i, alpha_j, omega_j, lambda_i, endowments)
    objective.backward()
    optimizer.step()

    if (iteration+1) % 100 == 0:
        print(f"Iteration {iteration}: Objective = {-objective.item()}")


# Final optimal allocations
final_allocations = torch.nn.functional.softplus(x_i).detach().numpy()

# print("Total allocations per good:")
# print(final_allocations.sum(axis=0))
# print("Endowments:")
# print(endowments.numpy())

df_final_homog = pd.DataFrame(
    final_allocations,
    index = [f'Agent {i+1}' for i in range(final_allocations.shape[0])],
    columns = [f'Good {j+1}' for j in range(final_allocations.shape[1])]
)

print("\n Optimal allocations for Homogeneous Agents:")
print(df_final_homog)


Iteration 99: Objective = 45.15373229980469
Iteration 199: Objective = 93.82718658447266
Iteration 299: Objective = 161.97462463378906
Iteration 399: Objective = 243.71197509765625
Iteration 499: Objective = 335.66778564453125
Iteration 599: Objective = 436.1275634765625
Iteration 699: Objective = 544.0576171875
Iteration 799: Objective = 569.2916870117188
Iteration 899: Objective = 569.2935791015625
Iteration 999: Objective = 569.2957763671875

 Optimal allocations for Homogeneous Agents:
            Good 1     Good 2     Good 3
Agent 1   9.866802   9.997384  10.040535
Agent 2  10.101438   9.863719  10.038213
Agent 3  10.036505  10.143645   9.925999


### Case II: Heterogeneous Agents 
> $m = n = 3$

* Each agent $i$ has their own set of $\alpha_j, \omega_j^i$ parameters.
* Pareto weights $\lambda_i$ differ among agents.
* Resource constraint remains the same. Total endowment for each good is 30.

In [62]:
# Social planner's objective, with Heterogeneous Agents

def social_planner_objective(x, alpha_j, omega_j, lambda_i, endowments, penalty_weight = 1000):
    total_utility = 0
    total_allocations = torch.zeros(m)

    for i in range(n):
        x_i_softplus = torch.nn.functional.softplus(x[i]) # ensure nonneg allocations
        total_utility += lambda_i[i] * utility(x_i_softplus, alpha_j[i], omega_j[i])
        total_allocations += x_i_softplus

    # penalize if RC is violated, i.e. total allocations > endowments
    penalty = torch.sum(torch.clamp(total_allocations - endowments, min = 0) ** 2)

    return -total_utility + penalty_weight * penalty   # maximization, so take negative


In [69]:
# =============== DEFINE PARAMETERS ===============
alpha_j = torch.tensor([[1.0, 0.8, 1.2],  # agent 1
                        [1.1, 0.9, 1.3],  # agent 2
                        [0.9, 0.7, 1.1]])  # agent 3

omega_j = torch.tensor([[0.3, 0.5, 0.7], 
                        [0.4, 0.6, 0.8],  
                        [0.5, 0.4, 0.6]]) 

lambda_i = torch.tensor([0.9, 1.1, 1.0])  # Social weights


# initial allocations (using small positive values)
x_i = torch.rand((n, m), requires_grad = True)
#x_i = (torch.rand((n, m), requires_grad=True) * endowments / n).clone().detach().requires_grad_(True)

# Set up optimizer
optimizer = optim.Adam([x_i], lr = 0.001)

# Optimization
num_iterations = 10000
for iteration in range(num_iterations):
    optimizer.zero_grad()
    objective = social_planner_objective(x_i, alpha_j, omega_j, lambda_i, endowments)
    objective.backward()
    optimizer.step()

    if (iteration+1) % 1000 == 0:
        print(f"Iteration {iteration}: Objective = {-objective.item()}")


# Final optimal allocations
final_allocations = torch.nn.functional.softplus(x_i).detach().numpy()

# print("Total allocations per good:")
# print(final_allocations.sum(axis=0))
# print("Endowments:")
# print(endowments.numpy())

df_final_heterog = pd.DataFrame(
    final_allocations,
    index = [f'Agent {i+1}' for i in range(final_allocations.shape[0])],
    columns = [f'Good {j+1}' for j in range(final_allocations.shape[1])]
)

print("\n Optimal allocations for Heterogeneous Agents:")
print(df_final_heterog)

Iteration 999: Objective = 13.591890335083008
Iteration 1999: Objective = 29.935745239257812
Iteration 2999: Objective = 51.88038635253906
Iteration 3999: Objective = 77.37189483642578
Iteration 4999: Objective = 105.61894226074219
Iteration 5999: Objective = 136.32730102539062
Iteration 6999: Objective = 169.35321044921875
Iteration 7999: Objective = 204.5991668701172
Iteration 8999: Objective = 220.5120086669922
Iteration 9999: Objective = 226.55992126464844

 Optimal allocations for Heterogeneous Agents:
            Good 1     Good 2     Good 3
Agent 1   9.150754   9.897849   9.951900
Agent 2  10.499072  11.412535  11.162815
Agent 3  10.351420   8.690802   8.888128
